In [2]:
!pip install  transformers
!pip install  tokenizers
!pip install  sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00


In [1]:
!pip install evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00


In [3]:
# argparse makes it easier to write user friendly command line interfaces
import argparse
 # package for faster file name matching
import glob
 # makiing directories for data
import os
 # reading json files as the data is present in json files
import json
 # time module for calculating the model runtime
import time
 # Allows writing status messages to a file
import logging
 # generate random float numbers uniformly
import random
 # regex module for text
import re
 # module provides various functions which work on
 # iterators too produce complex iterators
from itertools import chain
from string import punctuation
 # pandas for data manipulation
import pandas as pd
 # numpy for array operations
import numpy as np
 # PyTorch
import torch
 # provides various classes representing file system paths
 # with appropriate semantics
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
 # splitting the data
from sklearn.model_selection import train_test_split
 # ANSII color formatting for ouput in terminal
from termcolor import colored
 # wrapping paragraphs into string
import textwrap
 # model checkpoints in pretrained model
from transformers import (
     AdamW,
     T5ForConditionalGeneration,
     T5Tokenizer,
     get_linear_schedule_with_warmup
 )
import evaluate
# Bleu


In [4]:
import json
with open('/content/sample_data/CUAD_v1.json', 'rb') as f:
        data = json.load(f)


In [6]:
data.keys()

dict_keys(['version', 'data'])

In [20]:
data['data'][0].keys()
type(data['data'][0].get('paragraphs')[0].get('context'))

str

In [5]:
import json
with open('/content/sample_data/CUAD_v1.json', 'rb') as f:
        data = json.load(f)

question_list=[]
for i in range(len(data['data'])):
  questions=data['data'][i].get('paragraphs')[0].get('qas')
  question_list.append(questions)
context_list=[]
for i in range(len(data['data'])):
  context=data['data'][i].get('paragraphs')[0].get('context')
  context_list.append(context)
questions=data['data'][0].get('paragraphs')[0].get('qas')

for k in range(len(question_list)):
          for j in range(len(question_list[k])):
              context=data['data'][k].get('paragraphs')[0].get('context')
              context_list.append(context)
len(context_list)
for i in range(len(context_list),27646):
  context_list.append(context_list[21419])

In [6]:
def extract_questions_and_answers(p):
   with open(p, 'rb') as f:
     data = json.load(f)
     question_list=[]
     question=[]
     for i in range(len(data['data'])):
        questions=data['data'][i].get('paragraphs')[0].get('qas')
        question_list.append(questions)
     context_list=[]
     for k in range(len(question_list)):
          for j in range(len(question_list[k])):
              questions.append(question_list[k][j])
              context=data['data'][k].get('paragraphs')[0].get('context')
              context_list.append(context)
     data_rows = []

     for m in range(len(questions)):
       questions1=questions[m].get('question')
       answers = questions[m].get('answers')
       for answer in answers:
              answer_text = answer.get('text')
              answer_start = answer.get('answer_start')
              answer_end = answer.get('answer_start') + len(answer_text)  #Gets the end index of each answer in the paragraph
              data_rows.append({
                 "question" : questions1,
                 "answer_text" : answer_text,
                 "answer_start" : answer_start,
                 "answer_end" : answer_end
             })
     return pd.DataFrame(data_rows)

In [7]:
import pandas as pd
df = extract_questions_and_answers('/content/sample_data/CUAD_v1.json')
df

,question,answer_text,answer_start,answer_end
0,Highlight the parts (if any) of this contract ...,ENDORSEMENT AGREEMENT,1157,1178
1,Highlight the parts (if any) of this contract ...,North,1482,1487
2,Highlight the parts (if any) of this contract ...,Andy North,1477,1487
3,Highlight the parts (if any) of this contract ...,Company,613,620
4,Highlight the parts (if any) of this contract ...,Golfers Incorporated,22132,22152
...,...,...,...,...
27641,Highlight the parts (if any) of this contract ...,"Furthermore, Depomed will undertake to direct ...",152006,152340
27642,Highlight the parts (if any) of this contract ...,King recognizes Depomed's title to the Depomed...,64934,65180
27643,Highlight the parts (if any) of this contract ...,King acknowledges and agrees that it shall not...,65181,65583
27644,Highlight the parts (if any) of this contract ...,Depomed acknowledges and agrees that it shall ...,66571,66967


In [8]:
df['context']=context_list

In [9]:
df

,question,answer_text,answer_start,answer_end,context
0,Highlight the parts (if any) of this contract ...,ENDORSEMENT AGREEMENT,1157,1178,EXHIBIT 10.6\n\n ...
1,Highlight the parts (if any) of this contract ...,North,1482,1487,Exhibit 10.26 CONFIDENTIAL TREATMENT HAS BE...
2,Highlight the parts (if any) of this contract ...,Andy North,1477,1487,Exhibit 10.16 SUPPLY CONTRACT Contract No: Dat...
3,Highlight the parts (if any) of this contract ...,Company,613,620,1 ...
4,Highlight the parts (if any) of this contract ...,Golfers Incorporated,22132,22152,Exhibit 1\n\nJOINT FILING AGREEMENT\n\nThe und...
...,...,...,...,...,...
27641,Highlight the parts (if any) of this contract ...,"Furthermore, Depomed will undertake to direct ...",152006,152340,ENDORSEMENT AGREEMENT entered into by and b...
27642,Highlight the parts (if any) of this contract ...,King recognizes Depomed's title to the Depomed...,64934,65180,ENDORSEMENT AGREEMENT entered into by and b...
27643,Highlight the parts (if any) of this contract ...,King acknowledges and agrees that it shall not...,65181,65583,ENDORSEMENT AGREEMENT entered into by and b...
27644,Highlight the parts (if any) of this contract ...,Depomed acknowledges and agrees that it shall ...,66571,66967,ENDORSEMENT AGREEMENT entered into by and b...


In [10]:
from torch.utils.data import Dataset, DataLoader, RandomSampler

class CUAD(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        data_row = self.data.iloc[idx]

        question = data_row['question']
        context = data_row["context"]
        answer = data_row['answer_text']



        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [11]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast
from torch.optim import Adam

TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.001)
Q_LEN = 32   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 4


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
num_rows = len(df)
split_index = num_rows // 3

df = df.iloc[:split_index]
len(df)

9215

In [42]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = CUAD(TOKENIZER, df, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=4, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=4, sampler=val_sampler)

In [25]:
import torch
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0
MODEL.train()
for batch in train_loader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()


    #Evaluation
MODEL.eval()
for batch in val_loader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )


        val_loss += outputs.loss.item()
        val_batch_count += 1



In [26]:
MODEL.save_pretrained("qa_model")
TOKENIZER.save_pretrained("qa_tokenizer")

('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
 'qa_tokenizer/added_tokens.json',
 'qa_tokenizer/tokenizer.json')

In [37]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).unsqueeze(0)
    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)
    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                            references=[ref_answer])


        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
    else:
        return predicted_answer

In [43]:
context=val_data.iloc[100]['context']
question=val_data.iloc[100]['question']
answer_text=val_data.iloc[100]['answer_text']
predict_answer(context,question,answer_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Context: 
 Exhibit 10.4   INTELLECTUAL PROPERTY AGREEMENT   This INTELLECTUAL PROPERTY AGREEMENT (this "Agreement" or "IPA"), effective as of this 30  day of June 2016 (the "Effective Date") among THE HERTZ CORPORATION, a Delaware corporation, with an address of 8501 Williams Road, Estero, Florida 33928 (hereinafter "THC"); HERTZ SYSTEM, INC., a Delaware corporation, with an address of 8501 Williams Road, Estero, Florida 33928, United States of America (hereinafter "HSI") and HERC RENTALS INC., a Delaware corporation, with an address of 27500 Riverview Center Blvd., Bonita Springs, Florida 34134, United States of America (hereinafter "HERC") (hereinafter referred to collectively as the "Parties" and individually as a "Party").   WITNESSETH   WHEREAS, both HSI and HERC are wholly-owned subsidiaries of THC, and THC is an indirect wholly-owned subsidiary of Hertz Global Holdings, Inc., a Delaware corporation ("HGH");   WHEREAS, THC is the owner of a unique plan or system (hereinafter the 

{'Reference Answer: ': 'November 7, 2017',
 'Predicted Answer: ': 'DISCRETION',
 'BLEU Score: ': {'google_bleu': 0.0}}

In [51]:
context='the murder took place in broaddaylight around 9 in the morning where the accused stabbed the victim with a knife in the chestand escaped'
question='how did the murder happen'
predict_answer(context,question)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'a slain woman was stabbed in the chest with a knife in'

'         '